In [ ]:
import requests
import datetime
import time

server_url = "http://localhost:5000/sensor_data"

# Funzione per generare timestamp con intervalli di 30 secondi per 5 minuti
def generate_timestamps():
    start_time = datetime.datetime.utcnow()
    end_time = start_time + datetime.timedelta(minutes=5)
    timestamps = []
    current_time = start_time
    while current_time <= end_time:
        timestamps.append(current_time.isoformat() + "Z")  # Formato ISO 8601 UTC
        current_time += datetime.timedelta(seconds=30)
    return timestamps

# Funzione per inviare dati sintetici

def send_synthetic_data():
    timestamps = generate_timestamps()
    
    for t in timestamps:
        payload = {"pressure_value": 200, "time_custom": t}
        response = requests.post(server_url, json=payload)
        print(f"Sent {payload} at {payload['time_custom']}, Response: {response.status_code}")
        time.sleep(0.01)

# Invia i dati sintetici
send_synthetic_data()
